In [1]:
import numpy as np
import xarray as xr
import pyremo as pr
import glob
from REMO2020_mod import open_mfdataset

In [2]:
# datapath
dpath = "./data_in/RADKLIM/" # for original data
#dpath = "./data_in/RADKLIM/EUR-11/" # For remapped to EUR-11

In [3]:
# results path
respath = "./data_in/model_data/histogram/"

In [4]:
# define star year and end year
ystart=2001
yend=2010

In [5]:
# we want to keep attributes when addid dataset
xr.set_options(keep_attrs=True)

In [6]:
# give parameters for the bins
bin_start = 0.0
bin_end = 140
bin_step = 1.0

In [7]:
# months for analysis
anmons = np.array([6,7,8])

In [8]:
for year in range(ystart,yend+1):
    # filenames
    filenames = glob.glob(dpath+"grid_newprec_rw_"+str(year)+"*.nc")
    # load data
    ds = open_mfdataset(filenames, parallel=False, chunks='auto')
    # get histogram
    hist, bin_edges = np.histogram(ds.prec.where(ds.time.dt.month.isin(anmons), drop=True), bins=np.arange(bin_start,bin_end+bin_step,bin_step))
    xr_tmp = xr.Dataset(
            data_vars=dict(varri=(["bins"],hist,dict(standard_name="RADKLIM"))),
            coords=dict(bins=(["bins"],bin_edges[:-1]+bin_step/2.))
    ).compute()
    if(ystart != yend): # more than one year
        if(year == ystart):
            collect_data = xr_tmp
        else:
            collect_data = collect_data + xr_tmp
    else: # only one year
        collect_data = xr_tmp
    # clear memory
    del ds
# write to file
collect_data.to_netcdf(respath+"RADKLIM_1h_precipitation_hist_000000_"+str(ystart)+"-"+str(yend)+".nc",mode="w") # original
#collect_data.to_netcdf(respath+"RADKLIM_1h_precipitation_hist_000001_"+str(ystart)+"-"+str(yend)+".nc",mode="w") # EUR-11
# clear memory
del collect_data